### Retrieve anomalies from lookout for metrics (L4M) for a specified period

The following example shows how you can retrieve historical time series of anomalies from lookout for metrics.  
This example assumes you already have a running anomaly detector in Lookout For Metrics. You can refer to the [documentation](https://docs.aws.amazon.com/lookoutmetrics/latest/dev/detectors-setup.html) on how this can be set up.
You will also need compute to run the code. E.g. An Amazon EventBridge rule that runs on a schedule to trigger a lambda function. You can refer to the [documentation](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-create-rule-schedule.html) on how this can be set up. 

#### Install required python packages

In [1]:
# !pip install boto3
# !pip install pandas

#### Define constants used in Lookout for metrics APIs

In [2]:
import boto3
import datetime
import pandas as pd

In [ ]:
region = '<enter your region here>' # specify the AWS region
l4m_client = boto3.client('lookoutmetrics',region_name=region)
lookback_period_in_days = 730
sensitivity_threshold = 50 # confidence level of anomalies, the higher the value, the less anomalies returned
max_results = 50 # maximum number of anomalies returned per API call
detector_arn = '<enter your detector arn here>' # you can get this value from the console once the anomaly detector is set up

#### Define a function that retrieves the historical anomalies  
Anomalies are only retrieved if the end dates of anomalies fall within the lookback period specified  
This function uses 2 apis, *[list_anomaly_group_summaries](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lookoutmetrics.html#LookoutMetrics.Client.list_anomaly_group_summaries)* and *[list_anomaly_group_time_series](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lookoutmetrics.html#LookoutMetrics.Client.list_anomaly_group_time_series)*.   
*list_anomaly_group_summaries* needs to be invoked first to retrieve the anomaly group id and metric.  
Both of these are required paramters to call *list_anomaly_group_time_series* which returns the time series of the specified metric.  


In [ ]:
def retrieve_anomalies(client,lookback_period_in_days,sensitivity_threshold,max_results,detector_arn):
    end_date = datetime.datetime.now() # set latest date of lookback window as the current date
    start_date = end_date - datetime.timedelta(days=lookback_period_in_days) # set earliest date of lookback window as (current date - lookback_period_in_days)
    earliest_end_date = datetime.datetime.now() # earliest_end_date gets updated every api call with the earliest date in the list of anomalies returned

    anomaly_list = []
    next_token = ''
    
    params = dict(
        AnomalyDetectorArn = detector_arn,
        SensitivityThreshold = sensitivity_threshold,
        MaxResults = max_results
    )

    while start_date < earliest_end_date:
        
        # retrieve anomalies' group ids, start time, end time, metric name from L4M
        if not next_token:
            response = client.list_anomaly_group_summaries(
                **params
            )
        else:
            response = client.list_anomaly_group_summaries(
                **params,NextToken=next_token
            )
        
        earliest_end_date_str = response['AnomalyGroupSummaryList'][-1]['EndTime'].split('T')[0]
        earliest_end_date = datetime.datetime.strptime(earliest_end_date_str,'%Y-%m-%d')
        
        anomaly_groups = [{'id':anomaly['AnomalyGroupId'],'start_time':datetime.datetime.strptime(anomaly['StartTime'].split('T')[0],'%Y-%m-%d'),'end_time':datetime.datetime.strptime(anomaly['EndTime'].split('T')[0],'%Y-%m-%d'),'metric_name':anomaly['PrimaryMetricName']} for anomaly in response['AnomalyGroupSummaryList']]
        anomaly_list += anomaly_groups

        try:
            next_token = response['NextToken']
        except:
            break

    anomaly_list.reverse() # reverse the list to have the earlier alerts come first and later alerts come last

    # filter off data that do not fall within the lookback period
    cutoff_index = None
    for idx,anomaly in enumerate(anomaly_list):
        if start_date <= anomaly['end_time']:
            cutoff_index = idx
            break
    if cutoff_index is not None:
        anomaly_filtered = anomaly_list[cutoff_index:]
    else:
        anomaly_filtered = []

    df = pd.DataFrame(columns=['id','metric','timestamp','value'])

    ids = []
    metrics = []
    timestamps = []
    values = []

    # retrieve time series values for metric
    for anomaly in anomaly_filtered:

        response = client.list_anomaly_group_time_series(
            AnomalyDetectorArn=detector_arn,
            AnomalyGroupId=anomaly['id'],
            MetricName=anomaly['metric_name'],
            MaxResults=max_results,
        )

        length_of_time_series = len(response['TimestampList'])

        ids += [anomaly['id']]*length_of_time_series
        metrics += [response['MetricName']]*length_of_time_series
        timestamps += [timestamp.split('Z')[0] for timestamp in response['TimestampList']]
        values += response['TimeSeriesList'][0]['MetricValueList']
    
    # return data in pandas dataframe format and remove all rows with null values
    d = {
        'id':ids,
        'metric':metrics,
        'timestamp':timestamps,
        'value':values
    }
    df = pd.DataFrame(data=d)
    df.loc[:,'timestamp'] = pd.to_datetime(df.loc[:,'timestamp'])
    return df.dropna()

#### Define constants used in SES API

In [ ]:
from botocore.exceptions import ClientError
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
ses_client = boto3.client('ses',region_name=region)
sender = '<your send from email address>'
recipient = '<your send to email address>'

#### Define a function that sends anomaly report through emails using Amazon SES  
The api used here is *[send_raw_email](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ses.html#SES.Client.send_raw_email)* as opposed to *[send_email](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ses.html#SES.Client.send_email)* as attachments are involved.

In [ ]:
def notify_ses(df):
    msg = MIMEMultipart()
    msg["Subject"] = "l4m anomaly report"
    msg["From"] = sender
    msg["To"] = recipient

    body = MIMEText("View attached file for anomalies detected")
    msg.attach(body)

    attachment = bytes(df.to_csv(index=False),encoding='utf-8')

    part = MIMEApplication(attachment)
    part.add_header("Content-Disposition",
                    "attachment",
                    filename='anomalies.csv')
    msg.attach(part)
    try:
        response = ses_client.send_raw_email(
            Source=sender,
            Destinations=[recipient],
            RawMessage={"Data": msg.as_string()}
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        print(f"Email sent! Message ID: {response['MessageId']}")

In [ ]:
df = retrieve_anomalies(l4m_client,lookback_period_in_days,sensitivity_threshold,max_results,detector_arn)

In [ ]:
notify_ses(df)